In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import timeit

In [ ]:
%run FrozenLake.ipynb

env = FrozenLakeEnv()

# n-step SARSA

In [ ]:
class nStepSARSA(object):
    def __init__(self, __env, __n, __maxNumEp):
        self.env = __env
        self.n = __n
        self.maxNumEp = __maxNumEp
        self.numEp = 0
        self.qValues = np.zeros((env.observation_space.n,self.env.action_space.n))
        self.storedStates = {}
        self.storedActions = {}
        self.storedRewards = {}
        self.endOfEp = False
        self.regret = []
        self.sumRewards = 0
        self.maxReward = 0


    def behavPolicy(self, currState, ep):
        numAction = self.env.action_space.n

        pValues = [ep/numAction] * (numAction)

        ag = np.argmax(self.qValues[currState])
        pValues[ag] = pValues[ag] + 1 - ep

        return np.random.choice(np.arange(len(pValues)), p=pValues)

    def resetStateActionRewards(self):
        self.storedStates = {}
        self.storedActions = {}
        self.storedRewards = {}
    
    def storeState(self, currState, t):
        self.storedStates[(t+1)%(self.n)] = currState

    def storeAction(self, currAction, t):
        self.storedActions[(t+1)%(self.n)] = currAction

    def storeReward(self, currReward, t):
        self.storedRewards[(t+1)%(self.n)] = currReward

    def takeCurrAction(self, currAction):
        nextState, nextReward, done, info = self.env.step(currAction)
        self.endOfEp = done

        return [nextState, nextReward]

    def updateQValues(self, tau, alpha, discReturn):
        sTau = self.storedStates[tau%(self.n)]
        aTau = self.storedActions[tau%(self.n)]

        self.qValues[sTau][aTau] += alpha * (discReturn - self.qValues[sTau][aTau])
        
    def findEpRegret(self, numEp):
        epRegret = self.maxReward * numEp - (self.sumRewards)
        self.regret.append(epRegret)
        
    def plotRegrets(self):
        plot = plt.plot([i for i in range(1,len(self.regret)+1)], self.regret)
        plt.setp(plot, 'color', 'orchid')
        plt.show()

    def doAlgorithm(self):
        
        for ep in range(self.maxNumEp):
            self.resetStateActionRewards()
            
            currState = self.env.reset()
            self.storeState(currState, -1)

            currAction = self.behavPolicy(currState, 1)
            self.storeAction(currAction, -1)

            tau = 0
            t = -1
            gamma = 1
            T = sys.maxsize
            
            epReward = 0 

            while tau < (T-1):
                t += 1
        
                if t < T:
                    
                    #Take action and observe next state and reward
                    nextState, nextReward = self.takeCurrAction(self.storedActions[t%(self.n)])
                    self.storeState(nextState, t)
                    self.storeReward(nextReward, t)
                    self.sumRewards += nextReward
                    epReward += nextReward

                    if self.endOfEp:
                        T = t+1

                    else:
                        nextAction = self.behavPolicy(nextState, 1/(ep+1))
                        self.storeAction(nextAction, t)

                tau = t - self.n + 1

                if tau >= 0:
                    discReturn = np.sum([gamma**(i-tau-1)*(self.storedRewards[i%(self.n)]) for i in range(tau+1, min(tau+(self.n),T))])

                    if tau+(self.n) < T:
                        discReturn += (gamma**(self.n))*(self.qValues[self.storedStates[(tau)%(self.n)]][self.storedActions[(tau)%(self.n)]])

                    self.updateQValues(tau, 0.1, discReturn)
                    
            if self.maxReward < epReward:
                self.maxReward = epReward
                
            self.findEpRegret(ep)

        return self.regret

In [ ]:
numEp = 400

start_time = timeit.default_timer()
n1 = 1
myNStepSATSA = nStepSARSA(env, n1, numEp)           
regret1 = myNStepSATSA.doAlgorithm()
elapsed_time = timeit.default_timer() - start_time
print(' \n {} pisodes completed in {:.2f}s for n=1'.format(numEp, elapsed_time))

start_time = timeit.default_timer()
n2 = 5
myNStepSATSA = nStepSARSA(env, n2, numEp)           
regret2 = myNStepSATSA.doAlgorithm()
elapsed_time = timeit.default_timer() - start_time
print(' \n {} pisodes completed in {:.2f}s for n=5'.format(numEp, elapsed_time))

start_time = timeit.default_timer()
n3 = 100
myNStepSATSA = nStepSARSA(env, n3, numEp)           
regret3 = myNStepSATSA.doAlgorithm()
elapsed_time = timeit.default_timer() - start_time
print(' \n {} pisodes completed in {:.2f}s for n=100'.format(numEp, elapsed_time))

# plot1, = plt.plot([i for i in range(len(regret1))], regret1)
# plt.setp(plot1, 'color', 'orchid')

# plot2, = plt.plot([i for i in range(len(regret2))], regret2)
# plt.setp(plot2, 'color', 'darkorchid')

# plot3, = plt.plot([i for i in range(len(regret3))], regret3)
# plt.setp(plot3, 'color', 'blue')

# plt.legend([plot2,plot3],["n = 5", 'n=100'])
# plt.title('Regret for different values of n')
# plt.show()

# SARSA(lambda)

In [ ]:
import numpy as np

def epGreedy(stateQValues, numAction, ep):
    pValues = [ep/numAction] * (numAction)
    
    ag = np.argmax(stateQValues)
    
    pValues[ag] = pValues[ag] + 1 - ep
   
    return pValues

def chooseBestAction(pValues):
    return np.random.choice(np.arange(len(pValues)), p=pValues)

def plotForMe(myInput):
    plot = plt.plot([(i) for i in range(1,len(myInput)+1)], myInput)
    plt.setp(plot, 'color', 'orchid')
    plt.show()


class SARSALambda(object):
    
    def __init__(self, __env, __lambda, __numEp):
        self.env = __env
        self.algoLambda = __lambda
        self.gamma = 1
        self.numEpisode = __numEp
        self.endOfEp = False
        
        self.currAction = -1
        self.currState = -1
        self.currEpSteps = 0
        
        self.totalRewards = 0
        self.maxReward = 0
        
        self.qValues = np.zeros((env.observation_space.n,self.env.action_space.n))
        self.eligibility = np.zeros((env.observation_space.n,self.env.action_space.n))
        self.allRegrets = []
        
    
    def takeCurrAction(self):
        nextState, stateReward, done, info = self.env.step(self.currAction)
        self.endOfEp = done
        
        return [nextState, stateReward]
    
    def updateQ(self, alpha, delta):
        self.qValues += delta * alpha * self.eligibility
        
    def findBestAction(self, ep, state):
        stateQValues = self.qValues[state]
        numActions = self.env.action_space.n
        pValues = epGreedy(stateQValues, numActions, ep)
        
        return chooseBestAction(pValues)
        
    def updateElig(self):
        self.eligibility = self.gamma * self.algoLambda * self.eligibility
        
    def goToNextState(self, nextState, nextAction):
        self.currState = nextState
        self.currAcion = nextAction
        self.currEpSteps += 1
        
    def calcEpRegret(self, epIndex):
        if self.maxReward == 0:
            regret = 98 * epIndex - self.totalRewards
        else:
            regret = self.maxReward * epIndex - self.totalRewards
        self.allRegrets.append(regret)
        
    def plotRegret(self):
        print(' \t Leaning curve for lambda=',self.algoLambda)
        plot1, = plt.plot([i for i in range(len(self.allRegrets))], self.allRegrets)
        plt.setp(plot1, 'color', 'orchid')
        plt.show()

    def doAlgorithm(self):
        
        for i in range(self.numEpisode):
            #Init state
            self.currState = self.env.reset()
            self.currEpSteps = 0
            self.eligibility = np.zeros((env.observation_space.n,self.env.action_space.n))
            epReward = 0
            ep = 1/(i+1)
            
            #Choosing action in state
            self.currAction = self.findBestAction(ep, self.currState)


            while self.currEpSteps < 100:
                
                #Take action and observe s' and r
                nextState, stateReward = self.takeCurrAction()
                self.totalRewards += stateReward
                epReward += stateReward
                
                #Choosing next action in next state
                nextAction = self.findBestAction(ep, nextState)
                
                #Find delta
                delta = stateReward + self.gamma * self.qValues[nextState][nextAction] - self.qValues[self.currState][self.currAction]
                
                #Update eligibility of selected action and state
                self.eligibility[self.currState][self.currAction] = 1

                #Update Q for all state-actions so far
                self.updateQ(0.1, delta)
                
                #Update eilibility trace for all state-actions so far
                self.updateElig()
                
                #Go to next state
                self.goToNextState(nextState, nextAction)
                
            if self.maxReward < epReward:
                self.maxReward = epReward
                
            self.calcEpRegret(i+1)    
            self.endOfEp = False


        
numEp = 1000
start_time = timeit.default_timer()
lambda1 = 1
mySARSALambda = SARSALambda(env, lambda1, numEp)
mySARSALambda.doAlgorithm()
mySARSALambda.plotRegret()
elapsed_time = timeit.default_timer() - start_time
print(' \n 1000 pisodes completed in {:.2f}s for lambda=1'.format(elapsed_time))


start_time = timeit.default_timer()
lambda2 = 0.3
mySARSALambda = SARSALambda(env, lambda2, numEp)
mySARSALambda.doAlgorithm()
mySARSALambda.plotRegret()
elapsed_time = timeit.default_timer() - start_time
print(' \n 1000 pisodes completed in {:.2f}s for lambda=0.3'.format(elapsed_time))


start_time = timeit.default_timer()
lambda3 = 0
mySARSALambda = SARSALambda(env, lambda3, numEp)
mySARSALambda.doAlgorithm()
mySARSALambda.plotRegret()
elapsed_time = timeit.default_timer() - start_time
print(' \n 1000 pisodes completed in {:.2f}s for lambda=0'.format(elapsed_time))

